# This Notebook enables you to call every Function in the MTurk Functions catalog

## First we will import the packages we need to use the API
You'll need to do this before calling any Function API

In [3]:
from mturk_crowd_beta_client import MTurkCrowdClient
from boto3.session import Session
import uuid

## Next we set up a session

This examples assume you have a local AWS profile called __'mturk-crowd-caller'__, but you can authenticate however you like, including by directly passing in your access key and secret key.

In [5]:
# my profile name happens to be mtf-caller
session = Session(profile_name='mturk-crowd-caller')

crowd_client = MTurkCrowdClient(session)

## Available APIs:

* [sentiment-analysis](#sentiment-analysis)
* [image-contains](#image-contains)
* [emotion-detection](#emotion-detection)
* [semantic-similarity](#semantic-similarity)
* [collect-utterance-text](#collect-utterance-text)
* [image-categorization](#image-categorization)
* [bounding-box](#bounding-box)

## <a id='sentiment-analysis'>sentiment-analysis</a>

Input:
{
 "text": "Everything is wonderful!"
}

Result: {
  "taskName": "my-task-name",
  "input": {
    "text": "Everything is wonderful!"
  },
  "problemDetails": null,
  "state": "processing",
  "result": null
}

When you create a Task using the sentiment-analysis API, you're automatically creating a Human Intelligence Task (HIT) on worker.mturk.com.  Here's an example of a sentiment analysis HIT.

![sentiment-analysis-HIT-example](sentiment-analysis-HIT-example.png)

### Create a Task

In [4]:
#set the function_name to the name of the API
function_name = 'sentiment-analysis'

In [5]:
# automatically generate a random task ID
task_name = 'my-test-task-' + uuid.uuid4().hex
print(task_name)

my-test-task-b631d4359c8a45169746a4ffc3bfc45b


In [6]:
# define the text that you want analyzed, up to 16k
text = 'The trip by @VP Pence was long planned. He is receiving great praise for leaving game after the players showed such disrespect for country!'

In [7]:
# create a single task with the input you specified above
put_result = crowd_client.put_task(function_name,
                             task_name,
                             {'text': text})
print('PUT response: {}'.format(
    {'status_code': put_result.status_code, 'task': put_result.json()}))

PUT response: {'status_code': 201, 'task': {'input': {'text': 'The trip by @VP Pence was long planned. He is receiving great praise for leaving game after the players showed such disrespect for country!'}, 'problemDetails': None, 'result': None, 'state': 'processing', 'taskName': 'my-test-task-b631d4359c8a45169746a4ffc3bfc45b'}}


### Get the result

Wait a few minutes before calling get_task to give Workers a chance to submit answers

In [8]:
get_result = crowd_client.get_task(function_name, task_name)

print('GET response: {}'.format(                     
    {'status_code': get_result.status_code, 'task': get_result.json()}))

GET response: {'status_code': 200, 'task': {'input': {'text': 'The trip by @VP Pence was long planned. He is receiving great praise for leaving game after the players showed such disrespect for country!'}, 'problemDetails': None, 'result': None, 'state': 'processing', 'taskName': 'my-test-task-b631d4359c8a45169746a4ffc3bfc45b'}}


### Creating multiple Tasks and Processing Results 

Next, we'll read from a CSV file that has multiple rows of text that needs to be analyzed for sentiment using the pandas library.  As a prerequisite you need to have installed the pandas library.

In [9]:
import pandas as pd

In [13]:
#read the input data from csv into a pandas DataFrame
data = pd.read_csv('./sentiment-analysis-inputs.csv')

In [14]:
# column 1 is the Task ID and column2 is the input data
data

,task_id,review_text
0,customer_review_1,I'll give it 3 stars only because they were on...
1,customer_review_2,I love these boots and these are my 2nd pair. ...
2,customer_review_3,I read many reviews regarding the brown ones a...


In [15]:
#loop through the rows and create a Task per row, write the put_task status in a new column
status_codes = []
states = []
for index, row in data.iterrows():
    put_result = crowd_client.put_task(function_name, row.task_id,{'text': row.review_text})
    status_codes.append(put_result.status_code)
data['status_codes'] = status_codes

In [16]:
data

,task_id,review_text,status_codes
0,customer_review_1,I'll give it 3 stars only because they were on...,201
1,customer_review_2,I love these boots and these are my 2nd pair. ...,201
2,customer_review_3,I read many reviews regarding the brown ones a...,201


We'll call get_task periodically until all of the Tasks reach the "completed" state.

In [18]:
# loop through the rows and get results, store the state and results in new columns

status_codes = []
responses = []
for index, row in data.iterrows():
    get_result = crowd_client.get_task(function_name, row.task_id)
    status_codes.append(get_result.status_code)
    responses.append(get_result.json())

GET response: {'status_code': 200, 'task': {'input': {'text': "I'll give it 3 stars only because they were only worth the price I got them for $38 for the size 6. I'm glad I didn't pay more than that. I'm afraid these boots will probably not last more than 1 season. The boot/shaft part looks very nice and good quality but the foot part fits and looks so cheap. The leather seems like it's going to crack/peel very quickly. I will wear them until they give out."}, 'problemDetails': None, 'result': {'sentiment': 'neutral'}, 'state': 'completed', 'taskName': 'customer_review_1'}}
GET response: {'status_code': 200, 'task': {'input': {'text': 'I love these boots and these are my 2nd pair. However, I doubt I will buy them again. They look AMAZING. They are absolutely COMFORTABLE. But after 1 season they are so worn down you would think i had it for years. They are very cheaply made and the material is awful. I wish I could find the same "look" made with quality materials for a similar price..w

In [19]:
data['status_codes'] = status_codes
data['state'] = [r['state'] for r in responses]
data['problem_details'] = [r['problemDetails'] for r in responses]
data['sentiment'] = [r['result']['sentiment'] for r in responses]

In [20]:
data

,task_id,review_text,status_codes,state,problem_details,sentiment
0,customer_review_1,I'll give it 3 stars only because they were on...,200,completed,None,neutral
1,customer_review_2,I love these boots and these are my 2nd pair. ...,200,completed,None,positive
2,customer_review_3,I read many reviews regarding the brown ones a...,200,completed,None,positive


## <a id='image-contains'>image-contains</a>

Input: `{"image": {"url": "https://www.mturk.com/media/butterbean.jpg"}, "target": {"label": "dog"} }`

Result: `{"containsTarget": true}`

Internal Only: we ask 2 - 5 Workers to get agreement.

In [ ]:
function_name = 'image-contains'

### Create a Task

In [ ]:
# automatically generate a random task ID
task_name = 'my-test-task-' + uuid.uuid4().hex
print(task_name)

In [ ]:
# the URL of the image that you want annotated
image_url = 'https://urbanedge.blogs.rice.edu/files/2016/02/midtown-15wd4ck.jpg'

### The type of thing we're looking for

In [ ]:
label = 'Pedestrians'

In [ ]:
# create a single task with the input you specified above
put_result = crowd_client.put_task(function_name,task_name,{'image': {'url': image_url}, 'target': {'label': label} })

print('PUT response: {}'.format(
    {'status_code': put_result.status_code, 'task': put_result.json()}))        

### Get the result

Wait a few minutes before calling get_task to give Workers a chance to submit answers

In [ ]:
get_result = crowd_client.get_task(function_name, task_name)

print('GET response: {}'.format(
{'status_code': get_result.status_code, 'task': get_result.json()}))

## <a id='emotion-detection'>emotion-detection</a>

This API determines the emotion of text.

Input:{"text": "First time ever winning all three fantasy leagues AND @Seahawks win!"}
Result: {"emotion": "joy"}

emotion is one of Joy, Anger, Fear, Sadness, Surprise, Disgust, or Neutral

Internal only: we ask up to 9 Workers to get agreement.

In [ ]:
function_name = 'emotion-detection'

### Create a Task

In [ ]:
# automatically generate a random task ID
task_name = 'my-test-task-' + uuid.uuid4().hex
print(task_name)

In [ ]:
# define the text that you want analyzed, up to 16k
text = 'Just realised that I have #280characters so now I can finally tweet this: "Daenerys Stormborn of the House Targaryen, First of Her Name, the Unburnt, Queen of the Andals and the First Men, Khaleesi of the Great Grass Sea, Breaker of Chains, and Mother of Dragons'

In [ ]:
# create a single task with the input you specified above
put_result = crowd_client.put_task(function_name,
                             task_name,
                             {'text': text})
print('PUT response: {}'.format(
    {'status_code': put_result.status_code, 'task': put_result.json()}))

### Get the result

Wait a few minutes before calling get_task to give Workers a chance to submit answers

In [ ]:
get_result = crowd_client.get_task(function_name, task_name)

print('GET response: {}'.format(                     
    {'status_code': get_result.status_code, 'task': get_result.json()}))

## <a id=semantic-similarity>semantic-similarity</a>

Compare two text documents and rate them on how similar they are, on a scale between 0 and 1 where 1 is very similar.

Input: {"text1": "The sky is blue.", "text2": "The sky was the color of blue."}
Result: {"similarityScore": 0.75}

In [ ]:
function_name = 'semantic-similarity'

### Create a Task

In [ ]:
# automatically generate a random task ID
task_name = 'my-test-task-' + uuid.uuid4().hex
print(task_name)

In [ ]:
# define the text that you want analyzed, up to 16k
text1 = 'I\'m so hungry I could eat a horse'
text2 = 'I\'m hangry'

In [ ]:
#create a single task with the input you specified above
put_result = crowd_client.put_task(function_name,
                             task_name,
                             {'text1': text1, 'text2': text2})
print('PUT response: {}'.format(
    {'status_code': put_result.status_code, 'task': put_result.json()}))

### Get the result

Wait a few minutes before calling get_task to give Workers a chance to submit answers

In [ ]:
get_result = crowd_client.get_task(function_name, task_name)

print('GET response: {}'.format(                     
    {'status_code': get_result.status_code, 'task': get_result.json()}))

## <a id='collect-utterance-text'>collect-utterance-text</a>

Given a context and an intention, provide what you would say, in text, in that situation.

input: {"context": "Someone recently bought a phone and it doesn't work", "intent": "They want to return a phone"}'
result: {"utterance": "The phone that I just bought stopped working.  I want to get a refund"}

Internal only: We ask only 1 Worker for each task.


In [6]:
function_name = 'collect-utterance-text'

### Create a Task

In [7]:
# automatically generate a random task ID
task_name = 'my-test-task-' + uuid.uuid4().hex
print(task_name)

my-test-task-7d7d7ca4827d44b6ac6d8d3ae9bf089d


In [ ]:
# provide the context and the intent
context = 'Someone is calling their doctor\'s office'
intent = 'I want to change an existing appointment'

In [ ]:
#create a single task with the input you specified above
put_result = crowd_client.put_task(function_name,
                             task_name,
                             {'context': context, 'intent': intent})
print('PUT response: {}'.format(
    {'status_code': put_result.status_code, 'task': put_result.json()}))

### Get the result

Wait a few minutes before calling get_task to give Workers a chance to submit answers

In [19]:
get_result = crowd_client.get_task('collect-utterance-text', 'my-test-task-d89398e187ad4847ab677855efa4fbaa')

print('GET response: {}'.format(                     
    {'status_code': get_result.status_code, 'task': get_result.json()}))

GET response: {'status_code': 200, 'task': {'input': {'context': 'They recieved a bill that is higher than expected', 'intent': 'They want to get bill reduced to expected amount'}, 'problemDetails': None, 'result': None, 'state': 'processing', 'taskName': 'my-test-task-d89398e187ad4847ab677855efa4fbaa'}}


## <a id='image-categorization'>image-categorization</a>

Given an image URL and a list of categories, determine which category the image best belongs to.

Input: {"image":{"url": "https://requester.mturk.com/assets/simon.jpg"},"categories": “categories”: [{“label”: “Plant”}, {“label”:”Animal”}, {“label”:”Bacteria”}, {“label”:”Fungi”}, {“label”:”Protists”}]

Result: {"category":"Animal"} 

In [ ]:
function_name = 'image-categorization'

### Create a Task

In [ ]:
# automatically generate a random task ID
task_name = 'my-test-task-' + uuid.uuid4().hex
print(task_name)

In [ ]:
# provide the context and the intent
image_url = 'https://www.abebooks.com/images/books/harry-potter/sorcerers-stone.jpg'
categories =  [{'label': 'Hardcover'}, {'label':'Paperback'}, {'label':'eBook'}, {'label':'Audio Book'}]

In [ ]:
#create a single task with the input you specified above
put_result = crowd_client.put_task(function_name,
                             task_name,
                            {'image': {'url': image_url},
                            'categories': categories})
print('PUT response: {}'.format(
    {'status_code': put_result.status_code, 'task': put_result.json()}))

### Get the result

Wait a few minutes before calling get_task to give Workers a chance to submit answers

In [ ]:
get_result = crowd_client.get_task(function_name, task_name)

print('GET response: {}'.format(                     
    {'status_code': get_result.status_code, 'task': get_result.json()}))

## <a id='bounding-box'> bounding-box </a>

Given an image, draw a box around particular objects, specified by labels.

input: {"image": {"url": "https://sierra-videos.s3.amazonaws.com/2017_0521/DJI_0001_frames_720h/1495411174030000000.jpg"}, "labels": [{"label": "car"}, {"label": "pedestrians"}]}

result: 

In [13]:
function_name = 'bounding-box'

### Create a Task

In [14]:
# automatically generate a random task ID
task_name = 'my-test-task-' + uuid.uuid4().hex
print(task_name)

my-test-task-b7e19cf572814b0fb8e447bd8ea1fe80


In [15]:
# provide the context and the intent
image_url = 'https://sierra-videos.s3.amazonaws.com/2017_0521/DJI_0001_frames_720h/1495411174030000000.jpg'
labels =  [{"label": "car"}, {"label": "pedestrians"}]

In [16]:
#create a single task with the input you specified above
put_result = crowd_client.put_task(function_name,
                             task_name,
                            {'image': {'url': image_url},
                            'labels': labels})
print('PUT response: {}'.format(
    {'status_code': put_result.status_code, 'task': put_result.json()}))

PUT response: {'status_code': 404, 'task': {'message': 'Not Found'}}


### Get the result

Wait a few minutes before calling get_task to give Workers a chance to submit answers

In [ ]:
get_result = crowd_client.get_task(function_name, task_name)

print('GET response: {}'.format(                     
    {'status_code': get_result.status_code, 'task': get_result.json()}))